In [ ]:
# rules of the game
"""
1) Equipment :
3 red dice for attack
3 blue dice for defense
44 risk cards 'including 2 jokers: LATER'and 42 territories.
14 mission cards: LATER
No limit of troop

2) preliminary set up of game
Choose of troops depending of number of players => 3 players = 35 troops.....
distribute all the territories randomly to the players and add exactly one troop of the player  on the territory.
put the remaining troops in the stock of each Player


Game for 2 players rules with neutral army: LATER
40 troops per Player
28 troops for the neutral army
14 territories for the neutral army
14 territories for each Player.
2 troops on the territories of the neutral army
1 troop on each territory of each player

3)Who goes first? ==> give a player number 

4)The game itself
    a)Additionnal set up of the game:
Player 1 add 1 troop on one of its territories 
Player 2 add 1 troop on one of its territories ....and so on until all the troops are on the boardgame.
    b) Let's begin the actual game
Phase 1: reinforcement
    Get on 1 troop for every 3 territories you own
    Get bonus troops if you own the whole continent. Depends on the continent.
    Get extra troops depending of the combinations of the symbols on 3 risk cards you play : 3 infantry: 4 troops, 3 cavalry: 6 troops, 3 artillery: 8 troops, one of each :10 troops. start with only one or 2 combinations.
    Get extra troops depending of the territories on the risk cards you play: 2 extra troops if you are the owner of the territory at the moment you play
   f 
Phase 2: Attack
    Announce which territory attacks which territory. Be sure you take a atttacking territories from the territories you are occupying, and that those 2 territories are
    actually neighbours.
    Check if the destination territory does not belong to the player and if the number of troops is higher than 1 'no empty territory rule'
    decide our many troops you want to commit for the attack
    make sure that the number of troops attacking is <4 LATER
    roll the 3 dices on both sides and compared the max of each side . The max wins and the min loses one troop . 
    Automatically roll the dices until there a strict difference between the dice.
    maybe LATER : If the player attacking wins one or more territories : he wins 1 'territory cards '. Max 5 cars in the player hands==> he has to play them when its his turn and amount exceed 3.
    the player can keep attacking as long as he wants or until he cannot play 'loss or only territories with one troop'
    Display the board after every action

Phase 3: Manoeuver
    Can move only one group of troops from one territory to another . The territories have to belong to the player and no empty territory rule.
    'move function to be create to handle phase 3 and phase 2 too'
    
5) Engame
Either Player conquers at least 30 territories.
    
"""

In [7]:
class Country:
    
    def __init__(self,country_name):
        self.name=country_name
        self.color='White'
        self.nbtroops=0
     
    #udpate the color of the coutry with the color of the player which occupied it.
    def update_color_of_country(self,Player_obj):
        self.color=Player_ojb.color
        return self.color
    
    #udpate the number of troops which are in the country.
    def update_nb_troops(self,new_troop_nb):
        self.nbtroops=new_troop_nb
        return self.nbtroops
        

In [6]:
class Continent:
    
    def __init__(self,lst_countries_of_continent,bonus_of_continent):
        self.bonus=bonus_of_continent
        self.countries=lst_countries_of_continent
    
    # Function which check whether Player's color is on all the countries of the continent
    def is_eligible_to_bonus(self):
        pass
    
    # Function which provides the value of the bonus if the continent is fully occupied by Player.
    def get_value_of_bonus(self):
        return self.bonus
    
    
    
    

In [ ]:
#class MAP to create the boardgame with functions to check whether the boardgame created manually or automatically makes sense
class Map:
    def __init__(self,limits,lst_continents):
        self.frontiers=limits
        self.allcontinents=lst_continents
        self.allcountries=[Pays, for Pays in ]
    
    def get_all_continents(self):
        self.al
   
    def get_all_countries(self):
        self.allcountries=[Pays for Pays in]
    
    def Check_limits(self,limits,self.allcountries):
        
    

In [9]:
France=Country('France')
Italy=Country('Italy')
Spain=Country('Spain')
Morocco=Country('Morocco')
Senegal=Country('Senegal')

In [10]:
frontiers=[('France','Italy'), ('France','Spain'), ('Morocco','Spain')]

In [11]:
Europe=Continent([France,Spain,Italy],5) 
Africa=Continent([Morocco,Senegal],3)

In [ ]:
class Player:
    def __init__(self,player_id):
        self.id=player_id  # position of the player in the game
        self.occupied_territories=dict() # dictionnary with the territories owned and the number on each territory
        self.riskcards=[]
        self.missions=[] #LAter maybe
        self.stock=0 # counter for the reinforcement phase
        
    def get_occupied_territories(self,self.id):
         #provide the list of territories occupied by the player
        pass
    
    
        
        
        